In [8]:
import json
import urllib.request
import re
import pandas as pd

In [9]:
def get_json(title):
  baseurl = "https://marvel.fandom.com/api.php?"
  action = "action=query"
  title = "titles={}".format(title.replace(" ", "_"))
  content = "prop=revisions&rvprop=content&rvslots=*"
  dataformat ="format=json"

  query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)
    
  wikiresponse = urllib.request.urlopen(query)
  wikidata = wikiresponse.read()
  wikitext = wikidata.decode('utf-8')


  return json.loads(wikitext)

def displayWiki(wiki):
    code = str(list(wiki["query"]["pages"].keys())[0])
    title = wiki["query"]["pages"][code]["title"]
    content = wiki["query"]["pages"][code]["revisions"][0]["slots"]["main"]["*"]
    return title, content

In [10]:
teams = None
with open("teams.txt", "r") as f:
  teams = f.read().split("\n")[:-1]
teams

['Avengers (Earth-616)',
 'X-Men (Earth-616)',
 'Illuminati (Earth-616)',
 'Inhuman Royal Guard (Earth-616)',
 'Guardians of the Galaxy (Earth-616)',
 'Avengers (1,000,000 BC) (Earth-616)',
 'Sinister Six (Earth-616)',
 'Thunderbolts (Earth-616)',
 'Elders of the Universe (Earth-616)',
 'Young Avengers (Earth-616)',
 'Dark Avengers (Earth-616)',
 'Fantastic Four (Earth-616)',
 'Strategic Homeland Intervention, Enforcement and Logistics Division (Earth-616)',
 'Defenders (Earth-616)',
 'Hydra (Earth-616)',
 'Black Order (Earth-616)',
 'Cabal (Dark Illuminati) (Earth-616)',
 'Hand (Earth-616)',
 'Heralds of Galactus (Earth-616)',
 'Winter Guard (Earth-616)']

In [11]:
regex_links = r"\[\[(.*?)(?:|\|.*?)\]\]"

In [12]:
for team in teams:
  a = displayWiki(get_json(team))[1]
  b = re.split(r"\| First", a)[0]
  
  name = re.sub(" \(Earth-.*", "", team)
  leaders_raw = re.findall(r"Leaders.*?\| CurrentMembers", b, flags=re.DOTALL)[0]
  leaders = re.findall(regex_links, leaders_raw)
  
  current_member_raw = re.findall(r"\| CurrentMembers.*?\| FormerMembers", b, flags=re.DOTALL)[0]
  print(len(current_member_raw)>0)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [13]:
avengers = displayWiki(get_json("Avengers (Earth-616)"))[1]
len(avengers.split("First"))

3